<a href="https://colab.research.google.com/github/ttya16/llm-dev-practice/blob/main/notebooks/pt1_prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Dataset for Pretraining

## Find your source dataset
For here, we will use the dataset down below for practice.  
https://huggingface.co/datasets/fujiki/llm-japanese-dataset_wikinews?row=0&library=true

In [1]:
!pip install -q -U datasets
!pip install -q -U sentencepiece
!pip install -q -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 75.6 MB/s eta 0:00:00


In [2]:
import torch

from torch import nn
from torch.nn import functional as F

from datasets import load_dataset, load_from_disk
from datasets import Dataset

import numpy as np

from matplotlib import pyplot as plt

import time
import pandas as pd
import urllib.request

import csv

In [ ]:
%%time
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datasets/graelo/wikipedia.git
%cd wikipedia/data/20230601/ja/
!git lfs pull --include "train-*-of-0015.parquet"
%cd ../../../../

Cloning into 'wikipedia'...
remote: Enumerating objects: 2001, done.
remote: Total 2001 (delta 0), reused 0 (delta 0), pack-reused 2001
Receiving objects: 100% (2001/2001), 255.29 KiB | 6.90 MiB/s, done.
Resolving deltas: 100% (615/615), done.
/content/wikipedia/data/20230601/ja
/content
CPU times: user 2.77 s, sys: 387 ms, total: 3.15 s
Wall time: 3min 33s


In [ ]:
%cd wikipedia/data/20230901/ja/
!git lfs pull --include "train-*-of-0016.parquet"
%cd ../../../../

In [ ]:
# japanese wikipedia/wikinews

wikipedia_ds = load_dataset("wikipedia/data/20230601/ja/", data_files="train-*-of-0015.parquet", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
wikinews_ds = load_dataset("fujiki/llm-japanese-dataset_wikinews", split="train")

In [ ]:
wikinews_ds["input"][0]

'台湾の自由時報によると、3日、陳水扁総統夫人の呉淑珍氏と馬永成元総統府副秘書長、林徳訓総統府秘書らが国務機密費の流用容疑で、台北地方検察院により起訴された。台北地検は、総統夫人を主犯、総統府秘書であった馬氏、林氏の両名が共犯と認定した。読売新聞によれば、呉氏は2002年7月から2006年3月までの間に、機密費1,480万台湾ドル（日本円で約5,300万円）を横領し、指輪の購入など私的な目的に流用した疑いが持たれている。台北地検は陳総統本人も文書を偽造するなどして関与したと認定しているが、在任中の総統であるため、起訴することができなかった。これに対して、与党の民進党は呉総統夫人の起訴について中央評議会で審議するとともに、陳総統に対して国民の前で説明するよう要求した。自由時報によれば、野党の中国国民党と親民党は48時間以内に陳総統が辞任する事を求めた。もしそれが受け入れられなければ3回目の総統罷免案を立法院で動議する方針である。また民進党と政治的主張が近い台湾団結聯盟も、今回の罷免案に賛成する意向である。台湾団結聯盟の精神的指導者である李登輝・前総統も、12月9日に予定されている台北・高雄両直轄市の市長・市議会議員選挙において民進党候補を支援しないと述べている。一方、西日本新聞によれば、陳総統は5日夜、テレビで、「私的流用はなく、横領ではない」と述べ、容疑を否定した。また、「一審で有罪判決ならば総統を辞任する」と述べた。'

In [ ]:
# 事前学習に使うデータをテキストファイルへ書き出す
wikinews_ds.to_csv("wikinews.txt", columns=["input"], sep="\t", index=False, header=False)

Creating CSV from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

6328390

In [ ]:
wikipedia_ds.to_csv("wikipedia.txt", columns=["text"], sep="\t", index=False, header=False, quoting=csv.QUOTE_NONE, escapechar="\\")

Creating CSV from Arrow format:   0%|          | 0/1374 [00:00<?, ?ba/s]

6802686716

In [3]:
# Save raw data for easier access
from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_to_dataset_in_drive = "/content/drive/MyDrive/llm/dataset"
os.makedirs(path_to_dataset_in_drive, exist_ok=True)

# shutil.copyfile("wikinews.txt", os.path.join(path_to_dataset_in_drive, "wikinews.txt"))
# shutil.copyfile("wikipedia.txt", os.path.join(path_to_dataset_in_drive, "wikipedia.txt"))
wikinews_ds.save_to_disk(os.path.join(path_to_dataset_in_drive, "wikinews"))
# wikipedia_ds.save_to_disk(os.path.join(path_to_dataset_in_drive, "wikipedia"))

NameError: name 'wikinews_ds' is not defined

## Dataset Filtering
Basic Filtering to make llm better.





In [4]:
# load data if you have your datasets saved on disk
path_to_dataset_in_drive = "/content/drive/MyDrive/llm/dataset"
os.listdir(path_to_dataset_in_drive)

['wikipedia.txt',
 'wikinews',
 'wikipedia',
 'wikinews.txt',
 'wikinews_wikipedia.txt']

In [5]:
wikinews_ds = load_from_disk(os.path.join(path_to_dataset_in_drive, "wikinews"))
# wikipedia_ds = load_from_disk(os.path.join(path_to_dataset_in_drive, "wikipedia"))

In [6]:
wikinews_ds

Dataset({
    features: ['instruction', 'input', 'output', 'input_clean'],
    num_rows: 4265
})

### 1. Cleaning 正規化
neologdn  
https://github.com/ikegami-yukino/neologdn

In [7]:
!pip install -q -U neologdn
!pip install -q -U emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 10.3 MB/s eta 0:00:00


In [8]:
import neologdn
import emoji

def clean_text(text):
    text = emoji.demojize(text) # wikipedia系のデータには絵文字はない？
    text = neologdn.normalize(text)
    return text

def clean_sample(example):
    example["input_clean"] = clean_text(example["input"])
    return example


sample_text = "イェーイｲｪｲ今日も頑張るぞーーーーーーーーーーー！！！！！!!!"

clean_text(sample_text)

'イェーイイェイ今日も頑張るぞー!!!!!!!!'

In [9]:
wikinews_ds = wikinews_ds.map(clean_sample)

In [10]:
import difflib

for i in range(len(wikinews_ds["input_clean"])):
    if len(wikinews_ds["input_clean"][i]) < len(wikinews_ds["input"][i]):
        raw = wikinews_ds["input"][i]
        cleaned = wikinews_ds["input_clean"][i]
        diff = difflib.HtmlDiff()
        diff.make_file(raw, cleaned)
        with open("html_diff.html", "w") as f:
            f.write(diff.make_file(raw, cleaned))

        print(len(raw))
        print(len(cleaned))

        break

585
583


### 2. Remove duplicates
文字列がほぼ一致しているサンプルは除外する


https://github.com/google-research/deduplicate-text-datasets


https://github.com/ChenghaoMou/text-dedup


などある。  

ここでは一旦シンプルにpandas.DataFrame.drop_duplicateする。

In [ ]:
# !git clone https://github.com/google-research/deduplicate-text-datasets.git
# !cd deduplicate-text-datasets/ && cargo build
# !cd deduplicate-text-datasets/ && python3 scripts/make_suffix_array.py /content/drive/MyDrive/llm/dataset/wikinews/

In [13]:
wikinews_ds_df = pd.DataFrame(wikinews_ds)
pre_dedup_size = wikinews_ds_df.shape[0]
print(f"重複除去前：{wikinews_ds_df.shape[0]}件")
wikinews_ds_df.drop_duplicates(subset="input_clean", inplace=True)
print(f"重複除去後：{wikinews_ds_df.shape[0]}件")
print(f"重複除去率：{(pre_dedup_size - wikinews_ds_df.shape[0]) * 100 / pre_dedup_size :.3f}%")

wikinews_ds_df.reset_index(drop=True, inplace=True)
wikinews_ds_df.head()

重複除去前：4265件
重複除去後：3873件
重複除去率：9.191%


,instruction,input,output,input_clean
0,次のニュース記事にタイトルをつけてください。,台湾の自由時報によると、3日、陳水扁総統夫人の呉淑珍氏と馬永成元総統府副秘書長、林徳訓総統府...,台北地方検察院、吳淑珍総統夫人らを起訴—国務機密費の流用容疑,台湾の自由時報によると、3日、陳水扁総統夫人の呉淑珍氏と馬永成元総統府副秘書長、林徳訓総統府...
1,次のニュース記事にタイトルをつけてください。,岩波書店は約10年ぶりの大改訂となる「広辞苑 第六版」を1月11日に発売した。J-CASTに...,10年ぶりに広辞苑が大改訂—第六版発売,岩波書店は約10年ぶりの大改訂となる「広辞苑第六版」を1月11日に発売した。J-CASTによ...
2,次のニュース記事にタイトルをつけてください。,"4日、長野県安曇野市の長野県立こども病院は、心臓疾患を持って1,100gの低出生体重で生まれ...",1100ｇの男児の心臓手術に成功―長野,"4日、長野県安曇野市の長野県立こども病院は、心臓疾患を持って1,100gの低出生体重で生まれ..."
3,次のニュース記事にタイトルをつけてください。,ニッポン ニュース ネットワーク (NNN) によると27日、山形県庄内町の羽越線の脱線事故...,12月から速度規制解除—羽越線脱線事故,ニッポンニュースネットワーク(NNN)によると27日、山形県庄内町の羽越線の脱線事故でJR東...
4,次のニュース記事にタイトルをつけてください。,読売新聞によると、この手紙は1993年11月に当時小学一年生だった白髭奈津実さん（現在21歳...,15年前の手紙をカレイが届ける,読売新聞によると、この手紙は1993年11月に当時小学一年生だった白髭奈津実さん(現在21歳...


In [14]:
wikinews_ds = Dataset.from_pandas(wikinews_ds_df)
wikinews_ds

Dataset({
    features: ['instruction', 'input', 'output', 'input_clean'],
    num_rows: 3873
})

### 3. Remove toxic data

（実際に大規模言語モデル学習の文脈で使っていいかわからないが）  
OpenAIのModeration API or Perspective APIなどでやるのが一番楽か？

In [ ]:
!pip install -q -U openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

moderation = client.moderations.create(input="LLM開発は人類を滅亡させる")
print(moderation)

In [ ]:
wikinews_ds.to_csv("wikinews.txt", columns=["input"], sep="\t", index=False, header=False)

## Save Data for training Tokenizer

In [15]:
wikinews_ds.to_csv("wikinews.txt", columns=["input"], sep="\t", index=False, header=False)
shutil.copyfile("wikinews.txt", os.path.join(path_to_dataset_in_drive, "wikinews.txt"))

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

'/content/drive/MyDrive/llm/dataset/wikinews.txt'